# Model Comparison

Compares responses across:
- GROQ generative models (if GROQ_API_KEY is set)
- Extractive model (`scibert-extractive`) using the same retrieved context


In [ ]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd()))

from src.config import settings
from src.indexing.index_manager import IndexManager
from src.retrieval.retriever import RAGRetriever
from src.llm.orchestrator import LLMOrchestrator

import logging
logging.basicConfig(level=logging.INFO)

In [ ]:
pdf_path = "./data/sample_papers/sample_paper.pdf"
collection_name = "sample_paper_collection"
question = "What are the main contributions of the paper?"

index_manager = IndexManager(
    embedding_model=settings.EMBEDDING_MODEL,
    chunk_size=settings.CHUNK_SIZE,
    chunk_overlap=settings.CHUNK_OVERLAP,
    persist_dir=settings.CHROMA_PERSIST_DIR,
)

if Path(pdf_path).exists():
    index_manager.index_pdf(pdf_path, collection_name=collection_name)

vector_store = index_manager.get_vector_store()
embedder = index_manager.get_embedder()
vector_store.create_collection(collection_name)

retriever = RAGRetriever(
    vector_store=vector_store,
    embedder=embedder,
    top_k=5,
    similarity_threshold=settings.SIMILARITY_THRESHOLD,
)

retrieval = retriever.retrieve(question)
context = retrieval.get_context()
sources = retrieval.get_citations()

print("Sources:")
for s in sources:
    print("-", s)

print("\nContext preview:")
print(context[:500] + ("..." if len(context) > 500 else ""))

In [ ]:
orch = LLMOrchestrator()
models = orch.get_available_models()
print("Available models:", models)

# Compare only a small subset if many are present
candidate_models = []
for m in ["llama-3.3-70b-versatile", "mixtral-8x7b-32768", "gemma-7b-it", "scibert-extractive"]:
    if m in models:
        candidate_models.append(m)

print("Candidate models:", candidate_models)

In [ ]:
for m in candidate_models:
    print("\n" + "="*80)
    print("MODEL:", m)
    print("="*80)
    out = orch.generate(
        model_name=m,
        query=question,
        context=context,
        temperature=0.2,
        max_tokens=512,
        stream=False,
    )
    print(out)